# Guardian - Iteration 2


## Import Packages

In [11]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [12]:
iteration = 2

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2021-08-01",
    "to-date": "2022-01-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY2"),
}


In [13]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [14]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [15]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2021-08-01&to-date=2022-01-01&order-by=newest&api-key=934fc87f-0792-4733-b5ed-59d5aa136db0&page=1
status ok
total 32315
startIndex 1
pageSize 50
pages 647
orderBy newest
---- RUNTIME STATUS ---- 
(1/647) in 0:00:06.566783s
(2/647) in 0:00:05.773830s
(3/647) in 0:00:13.406903s
(4/647) in 0:00:04.671621s
(5/647) in 0:00:17.582123s
(6/647) in 0:00:08.752602s
(7/647) in 0:00:06.461810s
(8/647) in 0:00:04.552130s
(9/647) in 0:00:04.857531s
(10/647) in 0:00:06.145091s
(11/647) in 0:00:09.135444s
(12/647) in 0:00:15.841772s
(13/647) in 0:00:16.929200s
(14/647) in 0:00:05.187882s
(15/647) in 0:00:03.459682s
(16/647) in 0:00:03.557519s
(17/647) in 0:00:12.812444s
(18/647) in 0:00:07.206976s
(19/647) in 0:00:03.360886s
(20/647) in 0:00:06.543654s
(21/647) in 0:00:06.658169s
(22/647) in 0:00:05.930599s
(23/647) in 0:00:06.761017s
(24/647) in 0:00:05.097197s
(25/647) i

In [16]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [17]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(32315, 13) took 0:00:00.011188s

unfold_columns:
 shape=(32315, 49) took 0:00:08.938537s

drop_columns:
 shape=(32315, 31) took 0:00:00.020666s

create_csv:
 shape=(32315, 31) took 0:00:27.038471s

